In [267]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
import time

__file__ = os.getcwd()
__root__ = os.path.dirname(__file__)

In [268]:
DNAME_DATA = 'data'
fdir_data = os.path.join(__root__, DNAME_DATA)

DNAME_MODEL = 'model'
fdir_model = os.path.join(__root__, DNAME_MODEL)

# Corner Filter Labeling

In [269]:
def import_grid_pattern(fname):
    global fdir_data
    
    fpath = os.path.join(fdir_data, fname)
    _df = pd.read_excel(fpath, header=None, dtype=str, keep_default_na=False, na_values='NA')
    grid_patterns = _df.values.tolist()
        
    _id, grid_pattern = 0, []
    grid_patterns_dict = {}
    
    for row in grid_patterns:
        if '' in row:
            grid_patterns_dict[_id] = grid_pattern
            _id += 1
            grid_pattern = []
            continue
        else:
            grid_pattern.append(row)
    grid_patterns_dict[_id] = grid_pattern    
    
    return grid_patterns_dict

def export_grid_pattern_for_label(grid_patterns, fname_grid_patterns_for_label):
    global fdir_model
    
    grid_pattern_labels = ''
    for f in grid_patterns.values():
        grid_pattern_labels += '\n'.join([' '.join(row) for row in f]) + '\n--\n'

    fpath = os.path.join(fdir_model, fname_grid_patterns_for_label)
    with open(fpath, 'w', encoding='utf-8') as f:
        f.write(grid_pattern_labels)
        
    print('Export grid_pattern_for_labels at: {}'.format(fpath))

In [270]:
FNAME_GRID_PATTERN = 'grid_pattern.xlsx'
grid_patterns = import_grid_pattern(fname=FNAME_GRID_PATTERN)

FNAME_GRID_PATTERNS_FOR_LABEL = 'grid_pattern/before.txt'
export_grid_pattern_for_label(grid_patterns, FNAME_GRID_PATTERNS_FOR_LABEL)

Export grid_pattern_for_labels at: d:\IPM_Labs\Lab_workspace\STEAM\model\grid_pattern/before.txt


In [271]:
def import_corner_type_labeled(fname_corner_types_labeled):
    global fdir_model
    
    fpath = os.path.join(fdir_model, fname_corner_types_labeled)
    with open(fpath, 'r', encoding='utf-8') as f:
        labeled_types = [_type for _type in f.read().strip().split('SEP') if _type]
        
    filters = {}
    for row in labeled_types:
        corner_type, label = row.strip().split('--')
        corner_type_line = corner_type.strip().replace('\n', ' ')
        
        filters[corner_type_line] = label
        
    return filters

In [272]:
FNAME_GRID_PATTERNS_LABELED = 'grid_pattern/after.txt'

filters = import_corner_type_labeled(FNAME_GRID_PATTERNS_LABELED)

In [273]:
filters

{'0 0 0 0 1 0 0 0 0': '1',
 '1 0 0 0 1 0 0 0 0': '1',
 '0 0 0 1 1 0 0 0 0': '1',
 '1 0 0 1 1 0 0 0 0': '1',
 '0 0 0 0 1 1 0 0 0': '1',
 '1 0 0 0 1 1 0 0 0': '1',
 '0 0 0 1 1 1 0 0 0': '0',
 '1 0 0 1 1 1 0 0 0': '0',
 '0 0 0 0 1 0 0 1 0': '1',
 '1 0 0 0 1 0 0 1 0': '1',
 '0 0 0 1 1 0 0 1 0': '1',
 '1 0 0 1 1 0 0 1 0': '1',
 '0 0 0 0 1 1 0 1 0': '1',
 '1 0 0 0 1 1 0 1 0': '0',
 '0 0 0 1 1 1 0 1 0': '0',
 '1 0 0 1 1 1 0 1 0': '0',
 '0 0 0 0 1 0 0 0 1': '1',
 '1 0 0 0 1 0 0 0 1': '0',
 '0 0 0 1 1 0 0 0 1': '1',
 '1 0 0 1 1 0 0 0 1': '0',
 '0 0 0 0 1 1 0 0 1': '1',
 '1 0 0 0 1 1 0 0 1': '0',
 '0 0 0 1 1 1 0 0 1': '0',
 '1 0 0 1 1 1 0 0 1': '0',
 '0 0 0 0 1 0 0 1 1': '1',
 '1 0 0 0 1 0 0 1 1': '0',
 '0 0 0 1 1 0 0 1 1': '1',
 '1 0 0 1 1 0 0 1 1': '0',
 '0 0 0 0 1 1 0 1 1': '1',
 '1 0 0 0 1 1 0 1 1': '0',
 '0 0 0 1 1 1 0 1 1': '0',
 '1 0 0 1 1 1 0 1 1': '0',
 '0 0 0 0 1 0 1 0 0': '1',
 '1 0 0 0 1 0 1 0 0': '1',
 '0 0 0 1 1 0 1 0 0': '1',
 '1 0 0 1 1 0 1 0 0': '1',
 '0 0 0 0 1 1 1 0 0': '1',
 

# Case Study

In [274]:
def import_site(fname):
    global fdir_data
    
    fpath = os.path.join(fdir_data, fname)
    site = pd.read_excel(fpath, header=None, dtype=int).values
    return site

def pad_site(site):
    '''
    Attributes
    ----------
    site : np.array
    '''
    return np.pad(site, ((1,1),(1,1)))

def print_site(site):
    '''
    Attributes
    ----------
    site : np.array
    '''
    for row in site:
        print(' '.join([str(s) for s in row]))
        
def print_line(line):
    line_as_list = my_type_line.split(' ')    
    row_num = int(np.sqrt(len(line_as_list)))
    
    for row_idx in range(row_num):
        start = row_idx * row_num
        end = (row_idx+1) * row_num
        print(' '.join([str(e) for e in line_as_list[start:end]]))
        
def identify_surrounding(site, row_idx, col_idx):
    left_up = site[row_idx-1, col_idx-1]
    up = site[row_idx-1, col_idx]
    right_up = site[row_idx-1, col_idx+1]
    left = site[row_idx, col_idx-1]
    right = site[row_idx, col_idx+1]
    left_down = site[row_idx+1, col_idx-1]
    down = site[row_idx+1, col_idx]
    right_down = site[row_idx+1, col_idx+1]
    
    return [left_up, up, right_up, left, right, left_down, down, right_down]

def detect_vertex_by_filter(site, filters):
    row_num, col_num = site.shape
    site_padded = pad_site(site)

    vertex_map = deepcopy(site)
    for row_idx in range(row_num):
        for col_idx in range(col_num):
            row_idx_padded = row_idx + 1
            col_idx_padded = col_idx + 1
            me = site_padded[row_idx_padded, col_idx_padded]
            
            if me == 0:
                continue
            else:
                left_up, up, right_up, left, right, left_down, down, right_down = identify_surrounding(site_padded, row_idx_padded, col_idx_padded)
                my_type = [left_up, up, right_up, left, me, right, left_down, down, right_down]
                my_type_line = ' '.join([str(t) for t in my_type])

                is_vertex = filters[my_type_line]
                if is_vertex == '1':
                    vertex_map[row_idx, col_idx] = 2
                else:
                    continue

    return vertex_map

### Case #1 Rectangle

In [275]:
start = time.time()
FNAME_SITE = 'site_layout_sample/rectangle_10by10.xlsx'

site_rectangle = import_site(FNAME_SITE)
print_site(site_rectangle)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [276]:
vertex_map_rectangle = detect_vertex_by_filter(site_rectangle, filters)
print_site(vertex_map_rectangle)
end = time.time()
print("runtime : ",end -start)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

### Case #2 Truncated Rectangle